In [ ]:
# PostGreSQL data base
# Azure
# Google
# AWS
# Dockers
# Terraform

## Assistant API- Knowledge Retrieval
* External information can be provided to LLM
    * Fine Tune: Teaching the LLM new information (very expensve & requires expert skills)
    * Knowledge Retrieval: External informaion is converted into embeddings and saving in vector data bases. And embeddings related to the questions are provided in responses.
    * Function Calling

### Providing information to LLM 
* A file can be uploaded to LLM to 2 levels
    * At Assistant Level: Publically available and can be accessed by other assistants.
    * At Thread Level: Available only at particulat Thread Level

In [ ]:
%pip install -r requirements.txt

In [5]:
from dotenv import find_dotenv, load_dotenv

_ : bool = load_dotenv(find_dotenv())

In [6]:
from openai import OpenAI

client : OpenAI = OpenAI()

### Step-1: Upload a file and create assistant

In [ ]:
from openai.types.beta import Assistant

# upload a file with an "assistant's" purpose
# OpenAI will do upload, chunk, index, embedd, vector search and retrieve content from the file.

file = client.files.create(
    file = open("zia_profile.pdf", "rb"),
    purpose= "assistants"
)
print(file)

In [ ]:
# instruction to the assistant is general purpose and will be applicable to all the messages to this assistant

assistant : Assistant = client.beta.assistants.create(
    name="Student Support Assistant",
    instructions="You are astudent support chatbot. Use your knowledge base to best respond to student queries.",
    model="gpt-3.5-turbo-1106",
    tools=[{"type": "retrieval"}],
    file_ids=[file.id]      # connects file with this assistant
)

### Step-2: Create a Thread

In [ ]:
from openai.types.beta.thread import Thread

thread : Thread = client.beta.threads.create()

print(thread)

### Step-3: Add a message to thread

In [ ]:
from openai.types.beta.threads.thread_message import ThreadMessage

messsage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="When and which city Zia U. Khan was born?" # LLM will retrieve this information from our uploaded file
)

### Step-4: Run the Assistant

In [ ]:
# instructions to run are optional and specific to this task only

from openai.types.beta.threads.run import Run

run : Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Pakistani. The user is a student of PIAIC."
)

### Step-5: Check the Run Status

In [ ]:
run : Run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

print(run)

### Step-6: Display the Assistant's Response

In [ ]:
messages : list[ThreadMessage] = client.beta.threads.messages.list(
    thread_id=thread.id,
)

for m in reversed(messages.data):
    print(m.role + ": " + m.content[0].text.value)